In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tx
from sklearn.model_selection import train_test_split
from scipy.sparse.csgraph import laplacian
import torch
import torchvision
import torch.nn as nn
from torch import optim

In [27]:


class Graph:  # This class will trim the dataset to k mostly participating/active users
    def __init__(self, path_to_digg_votes, path_to_digg_friends, tau, test_split, alpha):
        self.digg_votes = np.array(pd.read_csv(path_to_digg_votes, header=None))
        self.digg_friends = np.array(pd.read_csv(path_to_digg_friends, header=None))
        self.tau = tau
        self.test_split = test_split
        self.alpha = alpha
    
    def top_k_nodes(self, dimension):
        unique_elements, counts_elements = np.unique(self.digg_votes[:,1], return_counts=True)
        dec_ix = np.argsort(counts_elements)[::-1]
        return unique_elements[dec_ix[:dimension]]
    
    def trim(self, dimension):
        self.nodes = self.top_k_nodes(dimension)
        self.num_of_nodes = len(self.nodes)
        self.nodes_to_ix = {}
        self.ix_to_node = {}
        for i in range(self.num_of_nodes):
            self.nodes_to_ix[self.nodes[i]] = i
            self.ix_to_node[i] = self.nodes[i]
        # Let us first calculate the Edge Score matrix
        self.S = np.zeros((self.num_of_nodes, self.num_of_nodes))
        for i in range(len(self.digg_friends)):
            if self.digg_friends[i][0] == 1:
                if self.digg_friends[i][2] in self.nodes:
                    if self.digg_friends[i][3] in self.nodes:
                        a = self.nodes_to_ix[self.digg_friends[i][2]]
                        b = self.nodes_to_ix[self.digg_friends[i][3]]
                        self.S[a,b] = 1
                        self.S[b,a] = 1
                        
        # Let us now calculate the C cascades
        self.C = []
        c = 1
        temp = []
        for i in tx(range(len(self.digg_votes))):
            if self.digg_votes[i][1] in self.nodes:
                if self.digg_votes[i][2] != c:
                    c+=1
                    if temp != []:
                        self.C.append(np.array(temp))
                    temp = []
#                 temp.append(np.array([self.nodes_to_ix[self.digg_votes[i][1]] , self.digg_votes[i][0], self.digg_votes[i][2]]))
                temp.append(np.array([self.nodes_to_ix[self.digg_votes[i][1]] , self.digg_votes[i][0]]))
        self.C_train, self.C_test, _, _ = train_test_split(self.C, self.C, test_size = self.test_split, random_state=42)
        self.num_of_cascades = len(self.C_train)
        #Now Let us calculate X
        self.X = [np.zeros((self.num_of_nodes, self.num_of_nodes)) for i in range(self.num_of_cascades)]
        self.P = [np.ones((self.num_of_nodes, self.num_of_nodes)) for i in range(self.num_of_cascades)]
        for i in tx(range(self.num_of_cascades)):
            for j in range(len(self.C_train[i])):
                for k in range(j+1, len(self.C_train[i])):
                    u = self.C_train[i][j][0]
                    v = self.C_train[i][k][0]
                    tu = self.C_train[i][j][1]
                    tv = self.C_train[i][k][1]
                    self.X[i][u,v] = np.exp(-(tu - tv)/self.tau)
                    if self.X[i][u,v] != 0:
                        self.P[i][u,v] = self.alpha
        # Let us now find the affinity matrix A
        self.A = np.zeros((self.num_of_nodes, self.num_of_nodes))
        for i in tx(range(self.num_of_nodes)):
            for j in range(i+1, self.num_of_nodes):
                c = self.count_affinity(i,j)
                if c!=0:
                    self.A[i,j] = c
                    self.A[j,i] = c
        
        # Let us now define the La and Ls- Laplacian matrix for cascading affinity matrix A and structural matrix S
        self.La = laplacian(np.array(self.A), normed=False)
        self.Ls = laplacian(np.array(self.S), normed=False)
        
    def count_affinity(self, u, v):
        c = 0
        for i in range(self.num_of_cascades):
            if u in self.C_train[i]:
                if v in self.C_train[i]:
                    c+=1
        return c
    
            
                
                
class AutoEncoder(nn.Module):
    def __init__(self, input_size):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_size, int(input_size/2)),
            nn.Sigmoid(),
            nn.Linear(int(input_size/2), int(input_size/4)),
            nn.Sigmoid(),
            nn.Linear(int(input_size/4), int(input_size/8)),  
        )
        self.decoder = nn.Sequential(
            nn.Linear(int(input_size/8), int(input_size/4)),
            nn.Sigmoid(),
            nn.Linear(int(input_size/4), int(input_size/2)),
            nn.Sigmoid(),
            nn.Linear(int(input_size/2), input_size),
            nn.Sigmoid(),  
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded
    
    
class Graph_embedding(nn.Module):
    def __init__(self, embedding_size, output_size):
        super(Graph_embedding, self).__init__()
        
        self.embedding = nn.Sequential(
            nn.Sigmoid(),
            nn.Linear(output_size, embedding_size),
            nn.Sigmoid(),
        )

    def forward(self, x):
        embd = self.embedding(x)
        return embd
      
     
            
def prob1(u,v, embd):
    b=(np.linalg.norm(embd[v]-embd[u]))**2
    a=1/(1+np.exp(b))
    return a

def prob_final(u,c, embd):
    prod=1;
    for i in range(math.ceil(0.1*c.shape[0])):
        prod=prod*(1-prob1(u,c[i,0], embd))
    return(1-prod)

def calc_score(nodes,c, embd):
    r_hat=[]
    noninfected_nodes=list(set(nodes)-set(c[:math.ceil(0.1*c.shape[0]),0]))
    for i in range(len(noninfected_nodes)):
        r_hat.append([noninfected_nodes[i],prob_final(noninfected_nodes[i],c, embd)])
    r_hat.sort(reverse=True,key=lambda x: (x[1]))
    return r_hat
    
def precision(r_hat,r):
    r_hat_set=set(r_hat)
    r_set=set(r)
    return (len(r_hat_set-r_set)/(len(r_hat_set)+1e-5))

def average_precison(nodes,c,k, embd):
    r_hat=calc_score(nodes,c, embd)
    prcsn=0;
    m=0;
    s=set(c[math.ceil(0.1*c.shape[0]):math.ceil(0.1*c.shape[0])+k,0])
    for j in range(k):
        if(np.array(r_hat)[j,0] in s):
            m+=1;
            prcsn=prcsn+precision(np.array(r_hat)[0:j,0],c[0:k,0])
    return (prcsn/k)

def mean_average_precision(nodes,C, embd):
    k=[50, 100, 200]
    mean_pre=[]
    for i in range(len(k)):
        mean_prcsn=0
        for j in range(len(C)):
            mean_prcsn += average_precison(nodes,C[j],k[i], embd)
        mean_pre.append(mean_prcsn/len(C))
    return mean_pre 
            
        
        

In [28]:
# Arguments
class Args:
    path_to_digg_votes = './Data/digg_votes1.csv'
    path_to_digg_friends = './Data/digg_friends.csv'
    batch_size = 10
    n_epoch = 10000
    path_to_save = '/'
    path_to_load = None
    phase = 'train'
    trim_dimension = 500
    tau = 100000 # By observations
    test_split = 0.2
    embedding_size = 50
    rho = 2
    lr = 0.01
    # Defining the losses coefficients
    alpha =  1
    beta = 0
    gamma = 0
    device = 'cuda:2'
    
args=Args()

In [22]:
path_to_digg_votes = args.path_to_digg_votes
path_to_digg_friends = args.path_to_digg_friends
batch_size = args.batch_size
n_epoch = args.n_epoch
path_to_save = args.path_to_save
phase = args.phase 
trim_dimension = args.trim_dimension
tau = args.tau
test_split = args.test_split
embedding_size = args.embedding_size
path_to_load = args.path_to_load
rho = args.rho
alpha = args.alpha
beta = args.beta
gamma = args.gamma
device = args.device
lr = args.lr


In [15]:
device = torch.device(device)

In [16]:
path_to_save = './node_500/'


S = torch.from_numpy(np.load(path_to_save + 'S.npy'))
X = torch.from_numpy(np.load(path_to_save + 'X.npy'))
P = torch.from_numpy(np.load(path_to_save + 'P.npy'))
# C = np.load(path_to_save + 'C.npy')
C_train = np.load(path_to_save + 'C_train.npy')
C_test = np.load(path_to_save + 'C_test.npy')
A = torch.from_numpy(np.load(path_to_save + 'A.npy'))
La = torch.from_numpy(np.load(path_to_save + 'La.npy'))
Ls = torch.from_numpy(np.load(path_to_save + 'Ls.npy'))





In [23]:
# Initializing the Graph and the autoencoders

input_size = X[0].shape[0]
num_of_cascades = len(X)
if path_to_load == None:
    AE = [AutoEncoder(input_size = input_size).to(device) for i in range(num_of_cascades)] #Set of all the autoencoders
    Embedding_net = Graph_embedding(embedding_size = embedding_size, output_size = int(input_size/8)).to(device)

In [24]:
params = list(Embedding_net.parameters())
for i in range(num_of_cascades):
    params += list(AE[i].parameters())
optimizer = optim.Adam(params, lr=lr)

In [29]:

# Training the model
no_of_iterations = int(X[0].shape[0]/batch_size)
for epoch in tx(range(1)):
    c = 0
    X_cap = []
    for i in range(no_of_iterations):
        if i+1 == no_of_iterations:
            for cascade in range(num_of_cascades):
                Input = X[cascade][c:].to(device)
                Input = Input.float()
                encoded, decoded = AE[cascade](Input)
                if i == 0:
                    X_cap.append(decoded)
                else:
                    X_cap[cascade] = torch.cat((X_cap[cascade], decoded), axis = 0)
                if cascade == 0:
                    temp = encoded
                else:
                    temp = temp + encoded
            embedding = Embedding_net(temp)
            c += batch_size
        else:
            for cascade in range(num_of_cascades):
                Input = X[cascade][c:c+batch_size].to(device)
                Input = Input.float()
                encoded, decoded = AE[cascade](Input)
                if i == 0:
                    X_cap.append(decoded)
                else:
                    X_cap[cascade] = torch.cat((X_cap[cascade], decoded), axis = 0)
                if cascade == 0:
                    temp = encoded
                else:
                    temp = temp + encoded
            embedding = Embedding_net(temp)
            c += batch_size
        
        if i == 0:
            Z = embedding
        else:
            Z = torch.cat((Z, embedding), axis = 0)
    loss1 = np.sum([torch.norm(torch.mul((X[i].to(device) - X_cap[i]), P[i].to(device))) for i in range(num_of_cascades)])
    loss2 = 2*torch.trace(torch.matmul(torch.matmul(Z.T,La.float().to(device)),Z))
#     loss3 = 2*torch.trace(torch.matmul(torch.matmul(Z.T,Ls.float().to(device)),Z))
#     l2_reg = torch.tensor(0.).to(device)
#     for i in range(num_of_cascades):
#         for param in AE[i].parameters():
#             l2_reg = l2_reg + torch.norm(param)
#     for param in Embedding_net.parameters():
#         l2_reg = l2_reg + torch.norm(param)
    loss = loss1 + alpha*loss2 #+ beta*loss3 + gamma*l2_reg
    # Backpropagation and then optimization
    optimizer.zero_grad()#Initially setting the gradient values to zero so backward() can find the gradient
    loss.backward()#backpropagate and then optimize
    optimizer.step()
    print('Epoch',epoch ,'Loss = ', loss.item())
    # for i in range(G.num_of_cascades):
    #   torch.save(AE[i],'Epoch_'+str(epoch)+'Loss'+str(loss)+'AE_'+str(i))
    # torch.save(Embedding_net, 'Epoch_'+str(epoch)+'Loss'+str(loss)+'emdd_net')
#     if loss.item() <= 4300:
#     np.save('./saved_embeddings_alpha1beta0/Z_'+str(embedding_size)+'_Epoch_'+str(epoch)+'_Loss_'+str(loss.item())+'.npy', np.array(Z.detach().cpu()))

            



Epoch 0 Loss =  4554.751566861018


FileNotFoundError: [Errno 2] No such file or directory: './saved_embeddings_alpha1beta0/Z_50_Epoch_0_Loss_4554.751566861018.npy'

tensor(10.5007, device='cuda:2', grad_fn=<MulBackward0>)

In [45]:
np.save('./saved_embeddings/Z_'+str(embedding_size)+'_Epoch_'+str(epoch)+'_Loss_'+str(loss.item())+'.npy', np.array(Z.detach().cpu()))